In [ ]:
# Example usage (replace 'path/to/your/directory' with your actual path)
master_directory = 'C:/Users/Hp/Desktop/IIT Research Docs/Movie Labeling/Augmented image_two_three/Val_TwoThree_Aug'
import os
import pandas as pd
# Function to count the number of files in a directory
def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

# Initialize an empty list to store directory names and counts
data = []

# Iterate through each item in the master directory
for item in os.listdir(master_directory):
    item_path = os.path.join(master_directory, item)
    if os.path.isdir(item_path):
        num_files = count_files(item_path)
        data.append({'Genre': item, 'Number_of_Files': num_files})

# Create a DataFrame from the list of dictionaries
val_df = pd.DataFrame(data)

In [ ]:
val_df.sort_values(by="Number_of_Files",ascending=False, ignore_index=True,inplace=True)

In [ ]:
val_df.to_csv("Images_by_Genre_Val_20k.csv")

In [ ]:
genres = [
    'Comedy',
    'Drama',
    'Action',
    'Crime',
    'Horror',
    'Romance',
    'Adventure',
    'Thriller',
    'Mystery',
    'Fantasy',
    'Sci-Fi',
    'Family',
    'Documentary',
    'Animation',
    'Music',
    'Biography',
    'History',
    'Sport',
    'War',
    'Musical'
]

In [ ]:
import os
import random
import pandas as pd

def select_images(primary_genre, reduction_ratio):
    # Step 1: Get all images from Primary Genre subdirectory
    master_folder = 'C:/Users/Hp/Desktop/IIT Research Docs/Movie Labeling/Augmented image_two_three/Val_TwoThree_Aug'
    primary_dir = os.path.join(master_folder, primary_genre)  # Assuming 'master_folder' is your master directory
    primary_images = os.listdir(primary_dir)
    
    # Step 2: Load total_files.csv to get total number of files in each subdirectory
    total_files_df = pd.read_csv('Images_by_Genre_Train.csv')  # Adjust path as per your setup
    
    # Step 3: Calculate x (total number of files in Primary Genre subdirectory)
    x = len(primary_images)
    
    # Step 4: Find subdirectories where the total number of files is less than (x - (x / reduction_ratio))
    less_images_directories = total_files_df[total_files_df['Number_of_Files'] < (x - (x / reduction_ratio))]['Genre'].tolist()
    
    # Step 5: Find subdirectories where the total number of files is greater than or equal to (x - (x / reduction_ratio))
    more_images_directories = total_files_df[total_files_df['Number_of_Files'] >= (x - (x / reduction_ratio))]['Genre'].tolist()
    
    # Step 6: Collect images for Case 1 (less_images_directories)
    result_df = pd.DataFrame(columns=['Image name', 'Genre'])
    
    for subdir in less_images_directories:
        subdir_path = os.path.join(master_folder, subdir)
        subdir_images = os.listdir(subdir_path)
        
        # Add all images from this subdir to result_df
        for image in subdir_images:
            result_df = result_df.append({'Image name': image, 'Genre': subdir}, ignore_index=True)
    
    # Step 7: Collect images for Case 2 (more_images_directories)
    primary_genre_set = set(primary_images)  # Set for quick lookup
    primary_genre_images_added = 0
    
    for subdir in more_images_directories:
        subdir_path = os.path.join(master_folder, subdir)
        subdir_images = os.listdir(subdir_path)
        
        # Add matching images from primary genre
        for image in subdir_images:
            if image in primary_genre_set and len(result_df[result_df['Genre'] == subdir]) < (x - (x / reduction_ratio)):
                result_df = result_df.append({'Image name': image, 'Genre': subdir}, ignore_index=True)
                primary_genre_images_added += 1
        
        # Check if we have collected enough images for this subdir
        if len(result_df[result_df['Genre'] == subdir]) >= (x - (x / reduction_ratio)):
            continue
        
        # Randomly pick images from this subdir until threshold is met
        random.shuffle(subdir_images)
        for image in subdir_images:
            if len(result_df[result_df['Genre'] == subdir]) < (x - (x / reduction_ratio)):
                result_df = result_df.append({'Image name': image, 'Genre': subdir}, ignore_index=True)
    
    # Step 8: Add all images from primary genre to result_df
    for image in primary_images:
        result_df = result_df.append({'Image name': image, 'Genre': primary_genre}, ignore_index=True)
    
    return result_df

In [ ]:
# Specify the folder where CSV files will be saved
output_folder = 'C:/Users/Hp/Desktop/IIT Research Docs/Movie Labeling/Validation CSV'  # Replace with your desired output folder path

# Iterate over each genre, process it, and save as CSV
for genre in genres:
    # Process the genre to get the DataFrame
    df_genre = select_images(genre,2)
    
    # Define the output CSV file path
    csv_filename = f"{genre}_data.csv"
    csv_path = os.path.join(output_folder, csv_filename)
    
    # Save the DataFrame as CSV
    df_genre.to_csv(csv_path, index=False)
    print(f"Saved {csv_filename} to {output_folder}")
